Cấu hình và phụ thuộc

Mục đích: thiết lập tham số huấn luyện và thư viện cần dùng.

In [8]:
import tensorflow as tf
print("TensorFlow:", tf.__version__)

from transformers import AutoTokenizer, TFAutoModel
MODEL_NAME = "dmis-lab/biobert-base-cased-v1.1"

tok = AutoTokenizer.from_pretrained(MODEL_NAME)
bert = TFAutoModel.from_pretrained(MODEL_NAME, from_pt=True)

print("Tokenizer OK:", type(tok).__name__)
print("Model OK:", type(bert).__name__)


TensorFlow: 2.16.2


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

Tokenizer OK: BertTokenizerFast
Model OK: TFBertModel


BƯỚC 1 — Đưa đầu vào (Input)

Mục tiêu

Đọc dữ liệu đã tiền xử lý (examples.parquet, vocab_meta.json).

Biến văn bản text thành token ID và attention mask mà BioBERT hiểu (tokenizer của BioBERT).

Tạo tf.data.Dataset cho train và validation (chỉ chuẩn bị đầu vào, chưa xây mô hình).

In [ ]:
# =========================
# BƯỚC 1 — ĐƯA ĐẦU VÀO (INPUT)  [CẬP NHẬT: thêm age & gender]
# =========================

# 1) Import các thư viện cần thiết
import json, random
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import GroupShuffleSplit
from transformers import AutoTokenizer

# 2) Cấu hình cơ bản cho việc tạo batch
MAX_LENGTH = 256   # độ dài tối đa của chuỗi token (cắt nếu dài hơn, pad nếu ngắn hơn)
BATCH_SIZE = 8     # kích thước lô (batch)
VAL_RATIO  = 0.15  # tỉ lệ dành cho validation
SEED = 42          # hạt giống ngẫu nhiên để tái lập kết quả

# 3) Đường dẫn dữ liệu đã tiền xử lý (đã bao gồm gender, age_at_admit)
DATA_ROOT = Path("..") / "data" / "proc"
PARQUET_PATH = DATA_ROOT / "examples.parquet"
VOCAB_PATH   = DATA_ROOT / "vocab_meta.json"

print("Đang dùng:")
print(" - PARQUET_PATH:", PARQUET_PATH.resolve())
print(" - VOCAB_PATH  :", VOCAB_PATH.resolve())

# 4) Kiểm tra sự tồn tại của file dữ liệu
if not PARQUET_PATH.exists():
    raise FileNotFoundError(
        f"Không tìm thấy {PARQUET_PATH}. Hãy chạy Notebook tiền xử lý để sinh file, "
        f"hoặc chỉnh lại đường dẫn cho đúng vị trí thực tế."
    )
if not VOCAB_PATH.exists():
    raise FileNotFoundError(
        f"Không tìm thấy {VOCAB_PATH}. Hãy kiểm tra Notebook tiền xử lý hoặc đường dẫn."
    )

# 5) Kiểm soát ngẫu nhiên để kết quả ổn định
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 6) Đọc dữ liệu bảng đã tiền xử lý và metadata nhãn
df = pd.read_parquet(PARQUET_PATH)
with open(VOCAB_PATH, "r") as f:
    meta = json.load(f)

icd_vocab        = meta["icd_vocab"]              # danh sách ICD-block (vocab)
lab_vocab_items  = meta["lab_vocab_items"]        # danh sách itemid xét nghiệm (vocab)
itemid_to_label  = {int(k): v for k, v in meta["itemid_to_label"].items()}

n_icd = len(icd_vocab)
n_lab = len(lab_vocab_items)

print(f"Số mẫu: {len(df)} | n_icd: {n_icd} | n_lab: {n_lab}")

# 7) Đảm bảo 2 cột nhãn là mảng float32 để dùng với BinaryCrossentropy
df["y_icd"] = df["y_icd"].apply(lambda a: np.asarray(a, dtype=np.float32))
df["y_lab"] = df["y_lab"].apply(lambda a: np.asarray(a, dtype=np.float32))

# 7.1) [MỚI] Chuẩn hoá demographics -> tabular features
# - gender -> one-hot (M/F/U) => 3 chiều
# - age_at_admit -> min-max [0..1] bằng cách chia 120 => 1 chiều
# Tổng số đặc trưng tabular: TAB_DIM = 4
def _normalize_gender(g):
    g = (str(g).upper().strip()[:1] if pd.notna(g) else "U")
    return g if g in ("M", "F", "U") else "U"

df["gender"] = df["gender"].apply(_normalize_gender)
df["age_at_admit"] = df["age_at_admit"].astype("float32")

# one-hot cho gender
gender_to_idx = {"M": 0, "F": 1, "U": 2}
idx = df["gender"].map(gender_to_idx).fillna(2).astype(int).values
gender_onehot = np.eye(3, dtype=np.float32)[idx]   # shape [N, 3]

# age chuẩn hoá (0..120) -> 0..1; thiếu thì 0
age_norm = (df["age_at_admit"].fillna(0.0).clip(0, 120) / 120.0).astype("float32").values.reshape(-1, 1)

# ghép thành vector tabular 4 chiều
tab_feats = np.concatenate([age_norm, gender_onehot], axis=1).astype("float32")  # [N, 4]
TAB_DIM = tab_feats.shape[1]

# 8) Chia train/validation theo subject_id (tránh leakage người bệnh)
groups = df["subject_id"].values
gss = GroupShuffleSplit(n_splits=1, test_size=VAL_RATIO, random_state=SEED)
train_idx, val_idx = next(gss.split(df, groups=groups))
train_df = df.iloc[train_idx].reset_index(drop=True)
val_df   = df.iloc[val_idx].reset_index(drop=True)

# cắt tương ứng tab_feats
train_tab = tab_feats[train_idx]
val_tab   = tab_feats[val_idx]

print("Số mẫu train:", len(train_df))
print("Số mẫu val  :", len(val_df))
print("TAB_DIM (age_norm + gender_onehot):", TAB_DIM)

# 9) Chuẩn bị tokenizer của BioBERT
try:
    tokenizer = tok  # tái dùng tokenizer đã nạp ở cell trước (nếu có)
except NameError:
    MODEL_NAME = "dmis-lab/biobert-base-cased-v1.1"
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 10) Hàm mã hóa 1 lô văn bản sang token ID & attention mask (đầu vào mô hình)
def encode_text_batch(text_list, max_length):
    enc = tokenizer(
        list(text_list),
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors=None
    )
    enc["input_ids"] = np.asarray(enc["input_ids"], dtype=np.int32)
    enc["attention_mask"] = np.asarray(enc["attention_mask"], dtype=np.int32)
    return enc

# 11) Tạo tf.data.Dataset theo kiểu generator (thêm 'tab_feats' vào X)
def make_tfds_from_df(frame, tab_array, batch_size, max_length):
    """
    Từ DataFrame gồm cột 'text', 'y_icd', 'y_lab' và mảng tab_array [N, TAB_DIM] tạo Dataset:
      - X = {
          "input_ids": [B, L],
          "attention_mask": [B, L],
          "tab_feats": [B, TAB_DIM],         # <<-- mới
        }
      - y = {
          "icd": [B, n_icd],
          "lab": [B, n_lab],
        }
    """
    texts = frame["text"].fillna("").tolist()
    y_icd = np.stack(frame["y_icd"].to_list()).astype(np.float32)
    y_lab = np.stack(frame["y_lab"].to_list()).astype(np.float32)

    def gen():
        N = len(texts)
        start = 0
        while start < N:
            end = min(start + batch_size, N)
            enc = encode_text_batch(texts[start:end], max_length=max_length)
            for i in range(end - start):
                yield (
                    {
                        "input_ids": enc["input_ids"][i],
                        "attention_mask": enc["attention_mask"][i],
                        "tab_feats": tab_array[start + i],   # <<-- mới
                    },
                    {
                        "icd": y_icd[start + i],
                        "lab": y_lab[start + i],
                    }
                )
            start = end

    output_signature = (
        {
            "input_ids": tf.TensorSpec(shape=(None,), dtype=tf.int32),
            "attention_mask": tf.TensorSpec(shape=(None,), dtype=tf.int32),
            "tab_feats": tf.TensorSpec(shape=(TAB_DIM,), dtype=tf.float32),  # <<-- mới
        },
        {
            "icd": tf.TensorSpec(shape=(n_icd,), dtype=tf.float32),
            "lab": tf.TensorSpec(shape=(n_lab,), dtype=tf.float32),
        },
    )

    ds = tf.data.Dataset.from_generator(gen, output_signature=output_signature)

    ds = ds.padded_batch(
        batch_size,
        padded_shapes=(
            {
                "input_ids": [MAX_LENGTH],
                "attention_mask": [MAX_LENGTH],
                "tab_feats": [TAB_DIM],          # cố định, không cần padding thêm
            },
            {
                "icd": [n_icd],
                "lab": [n_lab],
            },
        ),
        padding_values=(
            {
                "input_ids": tf.constant(0, tf.int32),
                "attention_mask": tf.constant(0, tf.int32),
                "tab_feats": tf.constant(0.0, tf.float32),
            },
            {
                "icd": tf.constant(0.0, tf.float32),
                "lab": tf.constant(0.0, tf.float32),
            },
        ),
        drop_remainder=False
    )
    return ds.prefetch(tf.data.AUTOTUNE)

# 12) Tạo train_ds và val_ds (đã kèm tab_feats)
train_ds = make_tfds_from_df(train_df, train_tab, BATCH_SIZE, MAX_LENGTH).shuffle(1024)
val_ds   = make_tfds_from_df(val_df,   val_tab,   BATCH_SIZE, MAX_LENGTH)

# 13) Kiểm tra nhanh một batch
for batch in train_ds.take(1):
    X, Y = batch
    print("Kích thước đầu vào (X):", {k: v.shape for k, v in X.items()})
    print("Kích thước nhãn (Y):   ", {k: v.shape for k, v in Y.items()})
    # gợi ý: text gốc vẫn xem từ train_df
    print("Ví dụ text gốc (rút gọn):", train_df.iloc[0]["text"][:120].replace("\n", " "))
    break


Đang dùng:
 - PARQUET_PATH: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/proc/examples.parquet
 - VOCAB_PATH  : /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/proc/vocab_meta.json
Số mẫu: 828 | n_icd: 200 | n_lab: 50
Số mẫu train: 717
Số mẫu val  : 111
Kích thước đầu vào (X): {'input_ids': TensorShape([8, 256]), 'attention_mask': TensorShape([8, 256])}
Kích thước nhãn (Y):    {'icd': TensorShape([8, 200]), 'lab': TensorShape([8, 50])}
Ví dụ text gốc (rút gọn): EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ORGAN) INDICATION: ___ year-old female with cirrhosis, jaundice. TECHNIQUE:


BƯỚC 2 — Xử lý trong BioBERT (tạo vector ngữ nghĩa)

Mục tiêu

Đưa batch input_ids + attention_mask vào BioBERT.

Lấy ra vector biểu diễn câu (dùng token đặc biệt [CLS]).

Kiểm tra kích thước và giá trị điển hình để xác nhận “dịch văn bản → ngôn ngữ số”.

In [ ]:
# =========================
# BƯỚC 2 — XỬ LÝ TRONG BIOBERT (CẬP NHẬT)
# =========================

import tensorflow as tf
from transformers import TFAutoModel

MODEL_NAME = "dmis-lab/biobert-base-cased-v1.1"
DROPOUT = 0.1  # dropout nhẹ cho CLS

# 2.1 Nạp backbone BioBERT (TF). Nếu đã có biến `bert` từ trước thì tái dùng.
try:
    _ = bert  # đã có sẵn
except NameError:
    bert = TFAutoModel.from_pretrained(MODEL_NAME, from_pt=True)

# 2.2 Inputs của encoder khớp với tf.data: CHỈ gồm input_ids & attention_mask
input_ids      = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")

# 2.3 Forward qua BioBERT, lấy CLS
bert_outputs = bert(input_ids, attention_mask=attention_mask, training=False)
cls_vec = bert_outputs.last_hidden_state[:, 0, :]  # [B, H]
cls_vec = tf.keras.layers.Dropout(DROPOUT, name="cls_dropout")(cls_vec)

# 2.4 Đóng gói encoder (text-only). Fusion với tab_feats sẽ làm ở bước sau.
encoder = tf.keras.Model(
    inputs={"input_ids": input_ids, "attention_mask": attention_mask},
    outputs={"cls": cls_vec},
    name="biobert_encoder_cls"
)

# 2.5 Kiểm tra nhanh 1 batch
for X_batch, Y_batch in train_ds.take(1):
    # CHỈ truyền 2 khóa mà encoder mong đợi, không đưa 'tab_feats'
    enc_inp = {
        "input_ids": X_batch["input_ids"],
        "attention_mask": X_batch["attention_mask"],
    }
    out = encoder(enc_inp, training=False)
    cls_batch = out["cls"].numpy()
    print("Kích thước vector CLS:", cls_batch.shape)
    print("8 số đầu của CLS[0]:", np.round(cls_batch[0][:8], 4).tolist())
    break


Kích thước vector CLS: (8, 768)
8 số đầu của CLS[0]: [0.13459999859333038, -0.023900000378489494, -0.17829999327659607, -0.04659999907016754, -0.436599999666214, -0.028200000524520874, 0.3319000005722046, 0.2379000037908554]


Ý nghĩa của kết quả

Bạn sẽ nhận được ma trận kích thước [BATCH_SIZE, HIDDEN] (thường HIDDEN = 768).

Mỗi hàng là “dấu vân tay số học” của 1 văn bản — chứa ngữ cảnh y khoa đã được BioBERT “đọc hiểu”.

Đây chính là đầu vào cho “phần ra quyết định” ở Bước 3.

BƯỚC 3 — Classifier head (Phần ra quyết định)

Mục tiêu

Gắn 2 “đầu ra quyết định” (Dense) lên vector CLS từ BioBERT:

Đầu ICD: dự đoán đa nhãn các ICD-block.

Đầu Lab: dự đoán đa nhãn các xét nghiệm sớm.

Mỗi đầu trả về logits (số thực âm/dương). Khi qua sigmoid sẽ thành xác suất 0–1 cho từng nhãn.

Kiểm tra kích thước đầu ra và xem thử top-k dự đoán trên một batch (chưa huấn luyện, chỉ để minh họa dòng chảy dữ liệu).

In [ ]:
# =========================
# BƯỚC 3 — CLASSIFIER HEAD (CẬP NHẬT: fusion CLS + tab_feats)
# =========================

import tensorflow as tf
import numpy as np

# 3.1 Lấy đầu ra CLS từ encoder (Bước 2)
cls_output = encoder.outputs[0]  # [batch, hidden_size], vd 768

# 3.2 Khai báo thêm input tabular để khớp với tf.data (Bước 1 đã tạo 'tab_feats')
tab_input = tf.keras.Input(shape=(TAB_DIM,), dtype=tf.float32, name="tab_feats")  # [batch, TAB_DIM]

# 3.3 Fusion: ghép CLS + tab_feats (có thể thêm MLP nhỏ để ổn định)
fused = tf.keras.layers.Concatenate(name="fuse_cls_tab")([cls_output, tab_input])  # [batch, 768+TAB_DIM]
fused = tf.keras.layers.Dropout(0.1, name="fuse_dropout")(fused)
fused = tf.keras.layers.Dense(512, activation="relu", name="fuse_dense")(fused)

# 3.4 Hai nhánh logits cho tác vụ đa nhãn (from_logits=True ở compile)
icd_logits = tf.keras.layers.Dense(n_icd, name="icd_logits")(fused)  # [batch, n_icd]
lab_logits = tf.keras.layers.Dense(n_lab, name="lab_logits")(fused)  # [batch, n_lab]

# 3.5 Mô hình đa nhiệm hoàn chỉnh (inputs = text + tabular)
multitask_model = tf.keras.Model(
    inputs={"input_ids": encoder.inputs["input_ids"],
            "attention_mask": encoder.inputs["attention_mask"],
            "tab_feats": tab_input},
    outputs={"icd": icd_logits, "lab": lab_logits},
    name="biobert_multitask_fusion"
)

# 3.6 Kiểm tra kiến trúc
multitask_model.summary(line_length=120)

# 3.7 Chạy thử một batch (X_batch đã có 'input_ids', 'attention_mask', 'tab_feats')
for X_batch, Y_batch in train_ds.take(1):
    logits = multitask_model.predict(X_batch, verbose=0)
    icd_log = logits["icd"]
    lab_log = logits["lab"]
    print("Kích thước icd_logits:", icd_log.shape)
    print("Kích thước lab_logits:", lab_log.shape)

    # Đổi sang xác suất để quan sát
    probs_icd = tf.sigmoid(icd_log).numpy()
    probs_lab = tf.sigmoid(lab_log).numpy()

    k = 5
    top_icd_idx = probs_icd[0].argsort()[-k:][::-1]
    top_lab_idx = probs_lab[0].argsort()[-k:][::-1]
    print("Top-5 ICD indices:", top_icd_idx.tolist())
    print("Top-5 ICD probs  :", np.round(probs_icd[0][top_icd_idx], 4).tolist())

    print("Top-5 Lab indices:", top_lab_idx.tolist())
    print("Top-5 Lab probs  :", np.round(probs_lab[0][top_lab_idx], 4).tolist())

    print("Top-5 ICD codes:", [icd_vocab[i] for i in top_icd_idx])
    print("Top-5 Lab itemids:", [lab_vocab_items[j] for j in top_lab_idx])
    break


Model: "biobert_multitask_heads"
________________________________________________________________________________________________________________________
 Layer (type)                       Output Shape                        Param #     Connected to                        
 input_ids (InputLayer)             [(None, None)]                      0           []                                  
                                                                                                                        
 attention_mask (InputLayer)        [(None, None)]                      0           []                                  
                                                                                                                        
 tf_bert_model_1 (TFBertModel)      TFBaseModelOutputWithPoolingAndCr   108310272   ['input_ids[0][0]',                 
                                    ossAttentions(last_hidden_state=(                'attention_mask[0][0]']            

Ý nghĩa của kết quả

Bạn sẽ thấy hai ma trận: icd_logits có kích thước [BATCH_SIZE, n_icd], lab_logits có kích thước [BATCH_SIZE, n_lab].

Sau khi áp sigmoid, bạn sẽ nhận được hai ma trận xác suất probs_icd, probs_lab cùng kích thước, mỗi cột là “một ô” xác suất mà bạn mô tả (ví dụ 0.85 cho I21, 0.92 cho Troponin…).

Đây chính là “phần ra quyết định” trước khi so sánh với nhãn thật ở Bước 4.

BƯỚC 4 — So sánh với nhãn thật (Ground truth)

Mục tiêu

Lấy dự đoán từ Bước 3 (logits → xác suất).

So sánh với nhãn thật y_icd, y_lab cho một mẫu trong validation:

Xem những nhãn nào mô hình dự đoán (theo ngưỡng 0.5) so với nhãn thật.

Xem Top-k (ví dụ k=5) nhãn có xác suất cao nhất và đối chiếu nhãn thật.

In [ ]:
# ===============================
# BƯỚC 4 — SO SÁNH VỚI NHÃN THẬT (CẬP NHẬT)
# ===============================

import numpy as np
import tensorflow as tf

# 1) Lấy một batch từ validation để so sánh
#    X_batch giờ gồm: input_ids, attention_mask, tab_feats
X_batch, Y_batch = next(iter(val_ds))

# 2) Chạy mô hình đa nhiệm (fusion từ Bước 3) để lấy logits
logits = multitask_model.predict(X_batch, verbose=0)
icd_log = logits["icd"]   # [BATCH_SIZE, n_icd]
lab_log = logits["lab"]   # [BATCH_SIZE, n_lab]

# 3) Đổi logits -> xác suất (0..1) bằng sigmoid để dễ "đọc"
probs_icd = tf.sigmoid(icd_log).numpy()
probs_lab = tf.sigmoid(lab_log).numpy()

# 4) Chọn một mẫu trong batch để nhìn chi tiết (ví dụ mẫu đầu tiên)
idx = 0
y_icd_true = Y_batch["icd"][idx].numpy()   # vector multi-hot thật [n_icd]
y_lab_true = Y_batch["lab"][idx].numpy()   # vector multi-hot thật [n_lab]

# 5) Chiến lược A: So sánh theo NGƯỠNG 0.5 (mặc định)
THR_ICD = 0.5
THR_LAB = 0.5

y_icd_pred_bin = (probs_icd[idx] >= THR_ICD).astype(int)
y_lab_pred_bin = (probs_lab[idx] >= THR_LAB).astype(int)

true_icd_idx = np.where(y_icd_true == 1)[0]
pred_icd_idx = np.where(y_icd_pred_bin == 1)[0]
hit_icd_idx  = np.intersect1d(true_icd_idx, pred_icd_idx)

true_lab_idx = np.where(y_lab_true == 1)[0]
pred_lab_idx = np.where(y_lab_pred_bin == 1)[0]
hit_lab_idx  = np.intersect1d(true_lab_idx, pred_lab_idx)

print("=== So sánh theo NGƯỠNG 0.5 (mẫu idx=0) ===")
print(f"ICD: thật={len(true_icd_idx)} | dự đoán={len(pred_icd_idx)} | đúng={len(hit_icd_idx)}")
print(f"LAB: thật={len(true_lab_idx)} | dự đoán={len(pred_lab_idx)} | đúng={len(hit_lab_idx)}")

# 6) Chiến lược B: So sánh theo TOP-K (ví dụ k=5)
K_ICD = 5
K_LAB = 5

top_icd_idx = probs_icd[idx].argsort()[-K_ICD:][::-1]
top_lab_idx = probs_lab[idx].argsort()[-K_LAB:][::-1]

print("\n=== So sánh theo TOP-K ===")
print("Top-5 ICD (index → mã → xác suất → có phải nhãn thật?):")
for j in top_icd_idx:
    code = icd_vocab[j]
    p = float(probs_icd[idx][j])
    is_true = int(y_icd_true[j])
    print(f"  idx={j:4d} | ICD={code:5s} | p={p:0.3f} | nhãn_thật={is_true}")

print("\nTop-5 Lab (index → itemid → tên → xác suất → có phải nhãn thật?):")
for j in top_lab_idx:
    itemid = int(lab_vocab_items[j])
    name = itemid_to_label.get(itemid, str(itemid))
    p = float(probs_lab[idx][j])
    is_true = int(y_lab_true[j])
    print(f"  idx={j:4d} | itemid={itemid} | {name} | p={p:0.3f} | nhãn_thật={is_true}")

# 7) (Tùy chọn) In danh sách nhãn đúng theo ngưỡng để hình dung
def map_icd_indices(indices):
    return [icd_vocab[i] for i in indices]

def map_lab_indices(indices):
    return [(int(lab_vocab_items[i]),
             itemid_to_label.get(int(lab_vocab_items[i]), str(lab_vocab_items[i])))
            for i in indices]

print("\nICD — nhãn_thật (mã):", map_icd_indices(true_icd_idx)[:20], "...")
print("ICD — dự_đoán_theo_ngưỡng (mã):", map_icd_indices(pred_icd_idx)[:20], "...")
print("ICD — giao_nhau_đúng (mã):", map_icd_indices(hit_icd_idx)[:20], "...")

print("\nLAB — nhãn_thật (itemid, tên):", map_lab_indices(true_lab_idx)[:10], "...")
print("LAB — dự_đoán_theo_ngưỡng (itemid, tên):", map_lab_indices(pred_lab_idx)[:10], "...")
print("LAB — giao_nhau_đúng (itemid, tên):", map_lab_indices(hit_lab_idx)[:10], "...")


=== So sánh theo NGƯỠNG 0.5 (mẫu idx=0) ===
ICD: thật=0 | dự đoán=90 | đúng=0
LAB: thật=0 | dự đoán=27 | đúng=0

=== So sánh theo TOP-K ===
Top-5 ICD (index → mã → xác suất → có phải nhãn thật?):
  idx=  53 | ICD=790   | p=0.796 | nhãn_thật=0
  idx=  72 | ICD=278   | p=0.782 | nhãn_thật=0
  idx=  14 | ICD=I48   | p=0.780 | nhãn_thật=0
  idx=   9 | ICD=Y92   | p=0.779 | nhãn_thật=0
  idx=  79 | ICD=412   | p=0.764 | nhãn_thật=0

Top-5 Lab (index → itemid → tên → xác suất → có phải nhãn thật?):
  idx=  11 | itemid=50902 | Chloride | p=0.844 | nhãn_thật=0
  idx=  44 | itemid=51274 | PT | p=0.807 | nhãn_thật=0
  idx=  37 | itemid=51116 | Lymphocytes | p=0.798 | nhãn_thật=0
  idx=  28 | itemid=51498 | Specific Gravity | p=0.688 | nhãn_thật=0
  idx=   7 | itemid=51279 | Red Blood Cells | p=0.683 | nhãn_thật=0

ICD — nhãn_thật (mã): [] ...
ICD — dự_đoán_theo_ngưỡng (mã): ['401', 'Z87', 'E87', 'Y92', 'V15', 'K21', 'I48', 'V45', 'N17', 'G47', 'Z86', '530', '427', 'N18', 'Z20', 'V10', 'Z85', 'F1

Ý nghĩa của kết quả

Bạn thấy rõ cách đối chiếu:

“Dự đoán theo ngưỡng” (≥ 0.5) vs. nhãn thật → đếm đúng/sai.

“Dự đoán Top-k” → xem 5 nhãn cao nhất và đánh dấu nhãn nào trùng nhãn thật.

Đây là bước không tính loss (chưa huấn luyện), chỉ so sánh để hiểu dòng chảy dự đoán ↔ nhãn.
(Loss sẽ thiết lập ở Bước 5.)

BƯỚC 5 — Tính sai số (Loss)

Mục tiêu

Khai báo hàm mất mát cho 2 nhánh đầu ra (ICD và Lab) đúng với thiết kế đa nhãn.

Biên dịch (compile) mô hình với optimizer/metrics, chưa huấn luyện.

Tính thử loss & AUPRC trên 1 batch để thấy mô hình đang ở mức “ngẫu nhiên” trước khi học.

In [ ]:
# ======================================
# BƯỚC 5 — TÍNH SAI SỐ (LOSS) & COMPILE (CẬP NHẬT)
# ======================================

import tensorflow as tf

# 1) Loss đa nhãn cho 2 nhánh (logits)
losses = {
    "icd": tf.keras.losses.BinaryCrossentropy(from_logits=True),
    "lab": tf.keras.losses.BinaryCrossentropy(from_logits=True),
}

# 2) Trọng số loss (tuỳ chỉnh nếu muốn ưu tiên 1 nhánh)
loss_weights = {"icd": 1.0, "lab": 1.0}

# 3) Metrics: AUPRC (phù hợp mất cân bằng nhãn)
metrics = {
    "icd": [tf.keras.metrics.AUC(curve="PR", multi_label=True, name="AUPRC")],
    "lab": [tf.keras.metrics.AUC(curve="PR", multi_label=True, name="AUPRC")],
}

# 4) Optimizer cho fine-tune
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)

# 🔒 4.1 Đóng băng backbone BioBERT (khi fusion có thêm tab_feats)
#     - Thử đóng băng trực tiếp biến `bert` nếu có
try:
    bert.trainable = False
except NameError:
    pass

#     - Phòng trường hợp backbone chỉ xuất hiện lồng trong model:
for sub in multitask_model.submodules:
    if "bert" in sub.name.lower():
        sub.trainable = False

# (Tuỳ chọn) in ra xác nhận trạng thái trainable của các module chứa 'bert'
print("Các module chứa 'bert' sau khi đóng băng:")
for sub in multitask_model.submodules:
    if "bert" in sub.name.lower():
        print(f"  - {sub.name}: trainable={sub.trainable}")

# 5) Compile mô hình (inputs gồm: input_ids, attention_mask, tab_feats)
multitask_model.compile(
    optimizer=optimizer,
    loss=losses,
    loss_weights=loss_weights,
    metrics=metrics
)

# 6) Đánh giá nhanh trên 1 batch validation (đầu vào đã có cả tab_feats)
X_batch, Y_batch = next(iter(val_ds))
results = multitask_model.evaluate(
    X_batch, Y_batch,
    verbose=0,
    return_dict=True
)

print("Kết quả trên 1 batch (trước khi train):")
for k, v in results.items():
    try:
        print(f"  {k}: {v:.4f}")
    except Exception:
        print(f"  {k}: {v}")


Backbone BioBERT trainable: False
Kết quả trên 1 batch (trước khi train):
  loss: 1.4856
  icd_logits_loss: 0.7190
  lab_logits_loss: 0.7666
  icd_logits_AUPRC: 0.0261
  lab_logits_AUPRC: 0.0000


BƯỚC 6 — Cập nhật (Training)

Mục tiêu

Huấn luyện toàn bộ mô hình: BioBERT + 2 đầu ra (ICD, Lab) bằng Adam.

Dùng EarlyStopping để dừng sớm nếu không cải thiện.

Lưu checkpoint tốt nhất theo chỉ số thẩm định (validation).

In [ ]:
# ==================================
# BƯỚC 6 — HUẤN LUYỆN (TRAINING) [CẬP NHẬT]
# ==================================

import tensorflow as tf
from pathlib import Path

EPOCHS = 3
OUT_DIR = Path("checkpoints_biobert_multitask")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# EarlyStopping theo AUPRC của nhánh ICD (đúng tên metric sau fusion: icd_AUPRC)
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_icd_AUPRC",
    mode="max",
    patience=1,
    restore_best_weights=True
)

# ModelCheckpoint lưu SavedModel tốt nhất theo val_icd_AUPRC
ckpt_path = OUT_DIR / "best"
ckpt_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=str(ckpt_path),
    monitor="val_icd_AUPRC",
    mode="max",
    save_best_only=True,
    save_weights_only=False,  # lưu SavedModel
    verbose=1
)

# (khuyến nghị) giảm LR khi metric không cải thiện
rlrop = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_icd_AUPRC",
    mode="max",
    factor=0.5,
    patience=1,
    min_lr=1e-6,
    verbose=1
)

# (tuỳ chọn) log lịch sử huấn luyện
csv_logger = tf.keras.callbacks.CSVLogger(OUT_DIR / "train_log.csv", append=False)

# Train
history = multitask_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[early_stop, ckpt_cb, rlrop, csv_logger],
    verbose=1
)

# Evaluate cuối
final_eval = multitask_model.evaluate(val_ds, return_dict=True, verbose=1)
print("\nĐánh giá cuối cùng trên validation:")
for k, v in final_eval.items():
    try:
        print(f"  {k}: {v:.4f}")
    except Exception:
        print(f"  {k}: {v}")

# Lưu bản final
final_path = OUT_DIR / "final"
multitask_model.save(final_path)
print(f"\nĐã lưu mô hình hiện tại tại: {final_path.resolve()}")
print(f"Checkpoint tốt nhất (theo val_icd_AUPRC) tại: {ckpt_path.resolve()}")


Epoch 1/3
     90/Unknown - 120s 1s/step - loss: 1.3712 - icd_logits_loss: 0.6643 - lab_logits_loss: 0.7069 - icd_logits_AUPRC: 0.0459 - lab_logits_AUPRC: 0.0076

2025-09-08 15:42:01.032424: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-09-08 15:42:18.976215: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]



Epoch 1: val_icd_logits_AUPRC improved from -inf to 0.03958, saving model to checkpoints_biobert_multitask/best
INFO:tensorflow:Assets written to: checkpoints_biobert_multitask/best/assets


INFO:tensorflow:Assets written to: checkpoints_biobert_multitask/best/assets


90/90 [==============================] - 171s 2s/step - loss: 1.3712 - icd_logits_loss: 0.6643 - lab_logits_loss: 0.7069 - icd_logits_AUPRC: 0.0459 - lab_logits_AUPRC: 0.0076 - val_loss: 1.2425 - val_icd_logits_loss: 0.6034 - val_lab_logits_loss: 0.6391 - val_icd_logits_AUPRC: 0.0396 - val_lab_logits_AUPRC: 0.0000e+00
Epoch 2/3
90/90 [==============================] - ETA: 0s - loss: 1.1564 - icd_logits_loss: 0.5652 - lab_logits_loss: 0.5911 - icd_logits_AUPRC: 0.0465 - lab_logits_AUPRC: 0.0053
Epoch 2: val_icd_logits_AUPRC did not improve from 0.03958
14/14 [==============================] - 18s 1s/step - loss: 1.2425 - icd_logits_loss: 0.6034 - lab_logits_loss: 0.6391 - icd_logits_AUPRC: 0.0396 - lab_logits_AUPRC: 0.0000e+00

Đánh giá cuối cùng trên validation:
  loss: 1.2425
  icd_logits_loss: 0.6034
  lab_logits_loss: 0.6391
  icd_logits_AUPRC: 0.0396
  lab_logits_AUPRC: 0.0000


2025-09-08 15:45:19.949599: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


INFO:tensorflow:Assets written to: checkpoints_biobert_multitask/final/assets


INFO:tensorflow:Assets written to: checkpoints_biobert_multitask/final/assets



Đã lưu mô hình hiện tại tại: /Users/lehoangkhang/Tài liệu/revita-sympdiag/notebooks/checkpoints_biobert_multitask/final
Checkpoint tốt nhất (theo val_icd_logits_AUPRC) tại: /Users/lehoangkhang/Tài liệu/revita-sympdiag/notebooks/checkpoints_biobert_multitask/best


Ý nghĩa của kết quả

loss, icd_loss, lab_loss trên validation sẽ giảm dần qua epoch; AUPRC sẽ tăng dần nếu dữ liệu/tiền xử lý ổn.

Sau khi huấn luyện, mô hình bắt đầu dự đoán hợp lý hơn (ít “tràn nhãn” ≥ 0.5, top-k sát thực tế hơn).